<a href="https://colab.research.google.com/github/sahuavi/Python-Codes/blob/main/Fault_dataset_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 23.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=f7900659eedf1d862bd9233a10545ed7174ae7ae9f4cbade4a52808c17f3372a
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


TASK:1

In [3]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("FaultDataset").getOrCreate()

# Load the dataset into a Spark DataFrame
df = spark.read.csv("FaultDataset.csv", header=True, inferSchema=True)

# Show the first 5 rows of the DataFrame
df.show(5)

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------------+
|        1|        2|        3|        4|        5|        6|        7|        8|        9|       10|       11|       12|       13|       14|       15|       16|       17|       18|       19|       20|fault_detected|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------------+
|0.3503125|0.3496875|     0.35|0.3459375|   0.3475|0.3459375| 0.341875|0.3434375|    0.355|0.3553125|0.3459375|   0.3525|   0.3575|0.3590625|  0.35875|0.3484375|0.3590625|     0.35|0.3559375|0.3490625|             0|
|0.5090625| 0.484375| 0.046875| 0.071875|     0.06|0.0634375|   0.0575|0.0546875|0.0559375| 0.058125|0.0628125| 0.065625|0.0640625|0

EDA

In [5]:
# Get the number of rows and columns in the DataFrame
print("Number of rows:", df.count())
print("Number of columns:", len(df.columns))

# Print the schema of the DataFrame
df.printSchema()

# Get summary statistics of the DataFrame
df.describe().show()

# Group the data by the "Fault" column and count the number of rows in each group
df.groupBy("fault_detected").count().show()


Number of rows: 9292
Number of columns: 21
root
 |-- 1: double (nullable = true)
 |-- 2: double (nullable = true)
 |-- 3: double (nullable = true)
 |-- 4: double (nullable = true)
 |-- 5: double (nullable = true)
 |-- 6: double (nullable = true)
 |-- 7: double (nullable = true)
 |-- 8: double (nullable = true)
 |-- 9: double (nullable = true)
 |-- 10: double (nullable = true)
 |-- 11: double (nullable = true)
 |-- 12: double (nullable = true)
 |-- 13: double (nullable = true)
 |-- 14: double (nullable = true)
 |-- 15: double (nullable = true)
 |-- 16: double (nullable = true)
 |-- 17: double (nullable = true)
 |-- 18: double (nullable = true)
 |-- 19: double (nullable = true)
 |-- 20: double (nullable = true)
 |-- fault_detected: integer (nullable = true)

+-------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+--------

TASK:2

In [7]:
# Import necessary libraries
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Split the data into training and testing sets
(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=42)

# Prepare the data for training by assembling the feature columns into a vector
assembler = VectorAssembler(inputCols=df.columns[:-1], outputCol="features")
trainingData = assembler.transform(trainingData).select("features", "fault_detected")
testData = assembler.transform(testData).select("features", "fault_detected")


In [9]:
# Train the Decision Tree classification model
dt = DecisionTreeClassifier(labelCol="fault_detected", featuresCol="features")

# Evaluate the performance of the model using Area Under ROC as the metric
evaluator = BinaryClassificationEvaluator(labelCol="fault_detected", metricName="areaUnderROC")

# Use cross-validation to tune the hyperparameters of the model
paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [2, 5, 10]).build()
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Fit the model to the training data
cvModel = cv.fit(trainingData)

# Make predictions on the test data and evaluate the performance of the model
predictions = cvModel.transform(testData)
auc = evaluator.evaluate(predictions)

# Print the area under ROC for the test data
print("Area Under ROC:", auc)


Area Under ROC: 0.8564325761668536


In [11]:
!pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 KB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/6

In [12]:
import mlflow
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Start an MLflow run
with mlflow.start_run():

  # Log the run parameters
  mlflow.log_param("maxDepth", 5)

  # Split the data into training and testing sets
  (trainingData, testData) = df.randomSplit([0.7, 0.3], seed=42)

  # Prepare the data for training by assembling the feature columns into a vector
  assembler = VectorAssembler(inputCols=df.columns[:-1], outputCol="features")
  trainingData = assembler.transform(trainingData).select("features", "fault_detected")
  testData = assembler.transform(testData).select("features", "fault_detected")

  # Train the Decision Tree classification model
  dt = DecisionTreeClassifier(labelCol="fault_detected", featuresCol="features", maxDepth=5)

  # Evaluate the performance of the model using Area Under ROC as the metric
  evaluator = BinaryClassificationEvaluator(labelCol="fault_detected", metricName="areaUnderROC")

  # Use cross-validation to tune the hyperparameters of the model
  paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [2, 5, 10]).build()
  cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

  # Fit the model to the training data
  cvModel = cv.fit(trainingData)

  # Make predictions on the test data and evaluate the performance of the model
  predictions = cvModel.transform(testData)
  auc = evaluator.evaluate(predictions)

  # Log the area under ROC for the test data
  mlflow.log_metric("auc", auc)

  # Log the model
  mlflow.spark.log_model(cvModel.bestModel, "model")
